In [1]:
# Notebook for preprocessing the Insurance Quote data based on BMI Business rules
# Import the libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
# Import the dataset
data = pd.read_csv('data/data.csv')
data.head()

,AppID,Ins_Age,Ins_Gender,Ht,Wt,IssueDate
0,56372,31,Male,510,185,NaN
1,34565,35,Male,510,205,NaN
2,57732,45,Female,510,125,NaN
3,87324,38,Male,503,175,NaN
4,12323,39,Female,600,252,NaN


In [3]:
# drop column names AppID and IssueDate as they are non value added
data.drop(['AppID', 'IssueDate'], axis='columns', inplace=True)

In [4]:
# Label Encoding the Gender column [Female: 0 and Male: 1]
encoder=LabelEncoder()
data.Ins_Gender=encoder.fit_transform(data.Ins_Gender)

In [5]:
def calculate_meters(Ht):
  '''
  Function for converting height into meters. 
  Height of applicant is in Ft with inches. The first digit refers to ft and following digits refers to inches
  Ex: 507 means 5 ft and 7 inches

  Parameters:
  Ht: Height in Feets and inches
  Return:
  meters: Output in meters
  '''
  feet = int(Ht[0:1]) # retreive height
  inches = int(Ht[1:2]) # retreive inches
  meters = (feet*12 + inches) * 0.0254 # convert to meters
  return meters

In [6]:
data['meters'] = data.apply(lambda row: calculate_meters(row['Ht'].astype(str)),axis=1)
data.drop(['Ht'], axis='columns', inplace=True)

In [7]:
# calculate BMI as per business rule. BMI = kg/(meter-suare)
data['Wt']=data['Wt']*0.453592 # convert applicant weight to kgs
data['BMI'] = round(data['Wt']/(data['meters']*data['meters']),2)

In [8]:
data

,Ins_Age,Ins_Gender,Wt,meters,BMI
0,31,1,83.914520,1.5494,34.96
1,35,1,92.986360,1.5494,38.73
2,45,0,56.699000,1.5494,23.62
3,38,1,79.378600,1.5240,34.18
4,39,0,114.305184,1.8288,34.18
...,...,...,...,...,...
95,35,1,124.737800,1.5494,51.96
96,27,1,65.770840,1.8288,19.67
97,35,0,86.182480,1.5240,37.11
98,38,0,65.317248,1.5494,27.21


In [9]:
def calculate_quote(record):
  '''
  Function to calculate insurance quote based on the BMI Business Rules

  Parameters:
  record: input record from the dataframe

  Return:
  quote_class: quote_class as per BMI rules
  '''
  # retreive age, gender and bmi values
  age = record['Ins_Age']
  gender = record['Ins_Gender']
  bmi = record['BMI']

  # calcualte insurance quote based on BMI Business rules 
  if ((age >= 18 and age <=39) and (bmi < 17.49 or bmi >38.5)):
    quote_class = 1
  elif ((age >= 40 and age <=59) and (bmi < 18.49 or bmi >38.5)):
    quote_class = 2
  elif ((age >= 60) and (bmi < 18.49 or bmi >45.5)):
    quote_class = 3
  else:
    quote_class = 0 
  
  return quote_class

In [10]:
data['quote'] = data.apply(lambda row: calculate_quote(row),axis=1)

In [11]:
data

,Ins_Age,Ins_Gender,Wt,meters,BMI,quote
0,31,1,83.914520,1.5494,34.96,0
1,35,1,92.986360,1.5494,38.73,1
2,45,0,56.699000,1.5494,23.62,0
3,38,1,79.378600,1.5240,34.18,0
4,39,0,114.305184,1.8288,34.18,0
...,...,...,...,...,...,...
95,35,1,124.737800,1.5494,51.96,1
96,27,1,65.770840,1.8288,19.67,0
97,35,0,86.182480,1.5240,37.11,0
98,38,0,65.317248,1.5494,27.21,0


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Ins_Age     100 non-null    int64  
 1   Ins_Gender  100 non-null    int64  
 2   Wt          100 non-null    float64
 3   meters      100 non-null    float64
 4   BMI         100 non-null    float64
 5   quote       100 non-null    int64  
dtypes: float64(3), int64(3)
memory usage: 4.8 KB


In [13]:
# Save processed data dataframes to new CSV files
data.to_csv("data/quote.csv", index=False)